In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
import datetime
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
start_date, end_date = '2018-10-22', datetime.datetime.today()

# 获取债券指数历史收益率

In [110]:
codes = 'CBA00111.CS, CBA00121.CS, CBA00131.CS, CBA00141.CS, CBA00151.CS'

In [111]:
data = w.wsd(codes, 'close', start_date, end_date)

In [112]:
data

.ErrorCode=0
.Codes=[CBA00111.CS,CBA00121.CS,CBA00131.CS,CBA00141.CS,CBA00151.CS]
.Fields=[CLOSE]
.Times=[20181022,20181023,20181024,20181025,20181026,20181029,20181030,20181031,20181101,20181102,...]
.Data=[[168.191,168.2038,168.2169,168.2309,168.2482,168.2993,168.3178,168.3168,168.3328,168.3378,...],[181.1973,181.2234,181.2847,181.3317,181.3727,181.5063,181.5647,181.5673,181.625,181.6271,...],[191.9484,191.9823,192.0654,192.1729,192.2332,192.5271,192.6177,192.6295,192.6616,192.6154,...],[196.5966,196.6772,196.8045,196.9654,197.0978,197.4548,197.5294,197.5509,197.7069,197.5774,...],[184.2893,184.4886,184.7905,184.9497,185.0183,185.4303,185.5143,185.5581,185.748,185.48,...]]

In [113]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [114]:
rdf = df.pct_change().dropna()

In [115]:
rdf.tail()

,CBA00111.CS,CBA00121.CS,CBA00131.CS,CBA00141.CS,CBA00151.CS
2019-11-18,0.000327,0.000611,0.001103,0.001837,0.002483
2019-11-19,0.000156,0.000423,0.000751,0.001100,0.001942
2019-11-20,0.000157,0.000292,0.000395,0.000341,-0.000066
2019-11-21,0.000106,0.000336,0.000608,0.000509,0.000459
2019-11-22,0.000000,0.000000,0.000000,0.000000,0.000000


# 获取债券基金历史收益率

In [116]:
fund_df = pd.read_excel(u'../data/债券基金池.xlsx')

In [117]:
fund_df

,代码,名称
0,002549.OF,嘉实稳祥纯债A
1,004544.OF,嘉实稳华纯债
2,070037.OF,嘉实纯债A
3,519723.OF,交银双轮动AB
4,000147.OF,易方达高等级信用债A
5,003358.OF,易方达7-10年国开行
6,070009.OF,嘉实超短债
7,000147.OF,易方达高等级信用债A


In [123]:
codes = fund_df[u'代码'].tolist()

In [124]:
data = w.wsd(codes, 'nav', start_date, end_date)

In [125]:
fdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [126]:
fdf.head()

,ErrorReport
2019-11-22,CWSDService: repeated windcodes.


In [121]:
rfdf = fdf.pct_change().dropna()

In [122]:
print rfdf.shape
print rdf.shape

(0, 1)
(266, 5)


# 非负线性优化求解

In [99]:
decomp = pd.DataFrame(index=codes, columns=[u'0-1', u'1-3', u'3-5', u'5-7', u'7-10'])
decomp.index.name = u'基金代码'

In [100]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

def decomposition(code):
    TOLERANCE = 1e-10
    n = rdf.shape[1]
    init_weights = np.array([1. / n] * n)
    X = rdf.values
    y = rfdf[code].values
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    opt_res = minimize(fun=objective,
                       x0=init_weights,
                       args=[X, y],
                       constraints=constraints,
                       tol=TOLERANCE,
                       options={'disp': True})
    
    return opt_res.x

In [101]:
decomposition(codes[0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.200004879502
            Iterations: 8
            Function evaluations: 56
            Gradient evaluations: 8


array([0.29892919, 0.27104051, 0.20644682, 0.12437038, 0.09921309])

In [102]:
for c in codes:
    weight = decomposition(c)
    decomp.loc[c] = weight

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.200004879502
            Iterations: 8
            Function evaluations: 56
            Gradient evaluations: 8
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.200292929104
            Iterations: 26
            Function evaluations: 182
            Gradient evaluations: 26
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.200400678538
            Iterations: 15
            Function evaluations: 105
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.200079887299
            Iterations: 8
            Function evaluations: 56
            Gradient evaluations: 8
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.201724953648
            Iterations: 14
            Function evaluations: 98
    

In [103]:
decomp

,0-1,1-3,3-5,5-7,7-10
基金代码,,,,,
002549.OF,0.298929,0.271041,0.206447,0.12437,0.0992131
004544.OF,0.472007,0.320093,7.80626e-18,-2.42455e-17,0.2079
070037.OF,0.411144,0.357205,0.231651,0,1.89733e-17
519723.OF,0.314593,0.294178,0.200451,0.111186,0.0795926
000147.OF,0.379125,0.338938,0.281937,0,1.77877e-19
003358.OF,-5.32514e-17,1.43079e-17,-1.26585e-16,2.77556e-17,1
070009.OF,0.461102,0.360512,0.178386,2.77556e-17,2.10932e-17


In [104]:
decomp[u'久期'] = decomp.dot(np.array([0.5, 2, 4, 6, 8]))

In [105]:
decomp[u'基金名称'] = fund_df[fund_df[u'代码'].isin(codes)][u'名称'].tolist()

In [106]:
decomp

,0-1,1-3,3-5,5-7,7-10,久期,基金名称
基金代码,,,,,,,
002549.OF,0.298929,0.271041,0.206447,0.12437,0.0992131,3.05726,嘉实稳祥纯债A
004544.OF,0.472007,0.320093,7.80626e-18,-2.42455e-17,0.2079,2.53939,嘉实稳华纯债
070037.OF,0.411144,0.357205,0.231651,0,1.89733e-17,1.84659,嘉实纯债A
519723.OF,0.314593,0.294178,0.200451,0.111186,0.0795926,2.85131,交银双轮动AB
000147.OF,0.379125,0.338938,0.281937,0,1.77877e-19,1.99519,易方达高等级信用债A
003358.OF,-5.32514e-17,1.43079e-17,-1.26585e-16,2.77556e-17,1,8,易方达7-10年国开行
070009.OF,0.461102,0.360512,0.178386,2.77556e-17,2.10932e-17,1.66512,嘉实超短债


In [107]:
decomp = decomp[[decomp.columns[-1]] + decomp.columns[:-1].tolist()]

In [108]:
decomp

,基金名称,0-1,1-3,3-5,5-7,7-10,久期
基金代码,,,,,,,
002549.OF,嘉实稳祥纯债A,0.298929,0.271041,0.206447,0.12437,0.0992131,3.05726
004544.OF,嘉实稳华纯债,0.472007,0.320093,7.80626e-18,-2.42455e-17,0.2079,2.53939
070037.OF,嘉实纯债A,0.411144,0.357205,0.231651,0,1.89733e-17,1.84659
519723.OF,交银双轮动AB,0.314593,0.294178,0.200451,0.111186,0.0795926,2.85131
000147.OF,易方达高等级信用债A,0.379125,0.338938,0.281937,0,1.77877e-19,1.99519
003358.OF,易方达7-10年国开行,-5.32514e-17,1.43079e-17,-1.26585e-16,2.77556e-17,1,8
070009.OF,嘉实超短债,0.461102,0.360512,0.178386,2.77556e-17,2.10932e-17,1.66512


In [109]:
decomp.to_excel(u'../data/债券基金收益率分解.xlsx')